In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib notebook
#
# ================== ЛИНЕЙНАЯ РЕГРЕССИЯ ===================================
#

# ЦЕЛЬ - ПРЕДСКАЗЫВАТЬ ВЫРУЧКУ КОМПАНИИ В НОВЫХ ГОРОДАХ В ЗАВИСИМОСТИ ОТ НАСЕЛЕНИЯ


# СЧИТАЕМ COST FUNCTION
def computeCost(X, y, theta):
    m = len(y)
    J = 1 / (2 * m) * sum((np.dot(X,theta) - y) ** 2)
    return J


# GRADIENT DESCENT
def gradient_descent(X, y, theta, alpha, num_iters):
    m = len(y); # number of training examples
    J_history = np.zeros(shape=(num_iters, 1))
    for iter in range(num_iters):
        theta = theta - alpha / m * np.dot(X.T, np.dot(X,theta) - y)   
        J_history[iter] = computeCost(X, y, theta)

    return [theta, J_history]


# СЧИТЫВАЕМ ДАННЫЕ ИЗ ФАЙЛА, В КОТОРОМ ДВА СТОБЦА
# ПЕРВЫЙ СТОЛБЕЦ - НАСЕЛЕНИЕ ГОРОДОВ
# ВТОРОЙ СТОБЕЦ - ВЫРУЧКА ИЗ КАЖДОГО ГОРОДА
data = pd.read_csv('ex1data1.txt', header=None)  

# РАЗДЕЛЯЕМ ДАННЫЕ
X = data.iloc[: , 0].values  # МАССИВ С НАСЕЛЕНИЕМ
X = X[:, np.newaxis]
y = data.iloc[: , 1].values  # МАССИВ С ВЫРУЧКОЙ
y = y[:, np.newaxis]

# ПОСТРОИМ ГРАФИК ДЛЯ ВИЗУАЛИЗАЦИИ ДАННЫХ
fig = plt.figure(figsize=(9.5,6.)) 
plt.plot(X, y, 'bo', label='Training data')  
plt.xlabel('Profit in $10,000s', fontsize='xx-large')
plt.ylabel('Population of City in 10,000s', fontsize='xx-large')


# ПОДГОТАВЛИВАЕМ МАТРИЦЫ И КОНСТАНТЫ
m = len(y)  # КОЛЛИЧЕСТВО ЭКЗЕМПЛЯРОВ
X = np.concatenate((np.ones(shape=(m, 1)), X), axis=1)  # ДОБАВЛЯЕМ СТОЛБЕЦ ЕДИНИЦ К X
theta = np.zeros(shape=(2, 1))  # ИНИЦИИРУЕМ СТОЛБЕЦ КОЭФИЦИЕНТОВ
iterations = 1500  # КОЛИЧЕСТВО ИТЕРАЦИЯ
alpha = 0.01  # СКОРОСТЬ ОБУЧЕНИЯ

computeCost(X, y, theta)
[theta, J_history] = gradient_descent(X, y, theta, alpha, iterations)
print(f'Theta computed from gradient descent: {theta[0][0]}, {theta[1][0]}')

# СТРОИМ ЛИНИЮ
plt.plot(X[:, 1], np.dot(X, theta), '-', label='Linear regression')
plt.legend(fontsize='x-large')

# ТЕПЕРЬ МОЖЕМ ПРЕДСКАЗЫВАТЬ ЗНАЧЕНИЯ
predict1 = np.dot([1, 3.5], theta)[0]
print(f'For population = 35,000, we predict a profit of {predict1 * 10000}')
predict2 = np.dot([1, 7], theta)[0]
print(f'For population = 70,000, we predict a profit of {predict2 * 10000}')

# СЧИТАЕМ COST FUNCTION ПРИ РАЗНЫХ КОЭФИЦИЕНТАХ (ДЛЯ ВИЗУАЛИЗАЦИИ)
theta0_vals = np.linspace(-10, 10, 100)
theta1_vals = np.linspace(-1, 4, 100)
J_vals = np.zeros(shape=(len(theta0_vals), len(theta1_vals)))
for i in range(len(theta0_vals)):
    for j in range(len(theta1_vals)):
        t = np.array([[theta0_vals[i]], [theta1_vals[j]]])
        J_vals[i, j] = computeCost(X, y, t)
        

# ВИЗУАЛИЗИРУЕМ COST FUNCTION
fig2 = plt.figure(figsize=(9.5,6.))
ax = fig2.add_subplot(111, projection='3d')
[theta0_vals, theta1_vals] = np.meshgrid(theta0_vals, theta1_vals)
surf = ax.plot_surface(theta0_vals, theta1_vals, np.transpose(J_vals), cmap = cm.coolwarm)
fig2.colorbar(surf, shrink = 1, aspect = 5)
ax.set_xlabel('theta 0')
ax.set_ylabel('theta 1')
ax.set_zlabel('Cost function')

# КОНТУРНЫЙ ГРАФИК
fig3 = plt.figure(figsize=(9.5,6.))
ax2 = fig3.add_subplot()
ax2.contour(theta0_vals, theta1_vals, np.transpose(J_vals))
ax2.plot(theta[0], theta[1], 'ro')

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#
# ================== ЛИНЕЙНАЯ РЕГРЕССИЯ С МНОГИМИ ПЕРЕМЕННЫМИ ===================================
#

# ЦЕЛЬ - ПРЕДСКАЗЫВАТЬ ЦЕНУ ДОМА В ЗАВИСИМОСТИ ОТ РАЗМЕРА И КОЛИЧЕСТВА КОМНАТ


# СЧИТАЕМ COST FUNCTION
def computeCost(X, y, theta):
    m = len(y)
    J = 1 / (2 * m) * sum((np.dot(X,theta) - y) ** 2)
    return J


# GRADIENT DESCENT
def gradient_descent(X, y, theta, alpha, num_iters):
    m = len(y); # number of training examples
    J_history = np.zeros(shape=(num_iters, 1))
    for iter in range( num_iters):
        theta = theta - alpha / m * np.dot(X.T, np.dot(X,theta) - y)   
        J_history[iter] = computeCost(X, y, theta)

    return [theta, J_history]


# ФУНКЦИЙ ДЛЯ НОРМИРОВАНИЯ ДАННЫХ
def feature_normalize(X):
    X = np.float64(X)
    X_norm = X
    mu = X.mean(axis = 0)   # СРЕДНИЕ ЗНАЧЕНИЯ ПРИЗНАКОВ
    sigma = X.std(axis = 0)  # СТАНДАРТНОЕ ОТКЛОНЕНИЕ
    
    for i in range(mu.shape[0]):
        X_norm[:, i] = (X[:, i] - mu[i]) / sigma[i]
    return [X_norm, mu, sigma]


# ФУНКЦИЯ ДЛЯ АЛГОРИТМА NORMAL EQUATION
def normal_equation(X, y):
    theta = np.dot(np.dot(np.linalg.pinv(np.dot(np.transpose(X),X)), np.transpose(X)), y);
    return theta


# СЧИТЫВАЕМ ДАННЫЕ ИЗ ФАЙЛАЦА
# X - ДАННЫЕ О ДОМАХ (1 СТОЛБЕЦ - РАЗМЕР   2 СТОЛБЕЦ - КОЛИЧЕСТВО КОМНАТ)
# y - ЦЕНЫ ДОМОВ
data = pd.read_csv('ex1data2.txt', header=None)
# РАЗДЕЛЯЕМ ДАННЫЕ
X = data.iloc[: , 0:2].values 
y = data.iloc[: , 2].values
y = y[:, np.newaxis]

# ПОСКОЛЬКУ НАШИ ДАННЫЕ ИМЕЮТ БОЛЬШОЙ РАЗБРОС, ПРОВЕДЕМ НОРМИРОВАНИЕ ПРИЗНАКОВ:
[X_norm, mu, sigma] = feature_normalize(X)
# ФУНКЦИИ, НАПИСАНЫЕ ДЛЯ ОДНОЙ ПЕРЕМЕННОЙ, ПОДХОДЯТ И ДЛЯ МНОГИХ ПЕРЕМЕННЫХ



# ИСПОЛЬЗУЕМ ДРУГОЙ АЛГОРИТМ - NORMAL EQUATION
X = np.concatenate((np.float64(np.ones(shape=(len(y), 1))), X), axis=1)  # Добавляем единицы
theta2 = normal_equation(X, y);

# НАЙДЕМ КОЭФИЦИЕНТЫ ГРАДИЕНТНЫМ СПУСКОМ
alpha = 0.1
num_iters = 400;
theta1 = np.zeros(shape=(3, 1))
X_norm = np.concatenate((np.float64(np.ones(shape=(len(y), 1))), X_norm), axis=1)
[theta1, J_history] = gradient_descent(X_norm, y, theta1, alpha, num_iters)

# СРАВНИМ ДВА РЕЗУЛЬТАТА, ПРЕДСКАЗАВ СТОИМОСТЬ ДОМА ПЛОЩАДЬЮ 1650 КВ.ФУТОВ У КОТОРОГО 3 КОМНАТЫ
price1 = np.dot(np.array([1, (1650 - mu[0]) / sigma[0], (3 - mu[1]) / sigma[1]]), theta1)
print(f'Predicted price of a 1650 sq-ft, 3 br house (using gradient descent):\n ${round(price1[0], 2)}')
price2 = np.dot(np.array([1, 1650, 3]), theta2)
print(f'Predicted price of a 1650 sq-ft, 3 br house (using normal equations):\n ${round(price2[0], 2)}')

Predicted price of a 1650 sq-ft, 3 br house (using gradient descent):
 $293081.46
Predicted price of a 1650 sq-ft, 3 br house (using normal equations):
 $293081.46
